# Hello

### Importar librerias

In [1]:
import kagglehub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import classification_report,accuracy_score
import json
import pickle

c:\Users\Ruben\OneDrive\Documentos\ITESM_IDM\Semestre 8\BloqueIngenieria\semana2\cloudComputing_act\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Extraer datos

In [15]:
#path = kagglehub.dataset_download("fedesoriano/company-bankruptcy-prediction")
#data = pd.read_csv(path+"/data.csv")

In [2]:
data = pd.read_csv("../data/pca_3_componentes.csv")

In [3]:
data.head(10)


,PC1,PC2,PC3,Bankrupt?
0,-7.338294,0.373294,-0.309014,1
1,-2.703713,-0.986346,-2.155617,1
2,-4.307059,-0.404700,-0.309801,1
3,-5.830736,-1.385669,-1.301105,1
4,-2.343197,-0.201748,0.806267,1
5,-6.681856,-0.774171,-0.927413,1
6,-5.855967,0.082141,-0.627971,0
7,-0.448640,0.203617,0.487619,0
8,-0.530649,-1.312022,-0.004166,0
9,-0.366075,0.735611,1.246965,0


In [4]:
data.shape

(6819, 4)

### Modelo

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(["Bankrupt?"], axis=1), data["Bankrupt?"], test_size=0.2, random_state=42)
X_train

,PC1,PC2,PC3
3759,-1.140608,0.480156,-0.759926
1782,-0.260267,0.572487,2.348049
5013,0.900051,-0.714695,-1.732955
5412,-1.186987,3.418958,3.690440
3066,-1.221979,0.844584,1.145609
...,...,...,...
3772,5.085515,1.215414,1.088639
5191,3.875070,0.616553,0.584459
5226,-0.117474,0.541386,1.877499
5390,4.052295,0.495118,0.217813


In [6]:
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [7]:
def sigmoid(x):
    return [1 / (1 + np.exp(-y)) for y in x]

y_pred = sigmoid(y_pred)

y_pred_sort = {i:y_pred[i] for i in range(len(y_pred))}
y_pred_sort = dict(sorted(y_pred_sort.items(), key=lambda x: x[1], reverse=True))

how_many_ones = y_test.value_counts()[1]

one_predictions = list(y_pred_sort.keys())[:how_many_ones]

final_preds = [1 if x in one_predictions else 0 for x in range(len(y_pred))]

In [8]:
print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1313
           1       0.41      0.41      0.41        51

    accuracy                           0.96      1364
   macro avg       0.69      0.69      0.69      1364
weighted avg       0.96      0.96      0.96      1364



In [9]:
X_test

,PC1,PC2,PC3
239,-2.508763,-3.236584,-2.634913
2850,0.554196,0.948523,0.697611
2687,3.204800,0.483832,0.528284
6500,-5.271985,2.967743,2.454384
2684,-1.350313,0.180171,-2.083013
...,...,...,...
1357,0.514038,1.313538,1.760316
3946,0.981786,-1.853898,0.566422
5491,-2.099497,0.530659,-0.590708
2112,-3.671366,1.539616,1.930818


In [10]:
accuracy = accuracy_score(y_test,final_preds)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9560117302052786


In [12]:
umbral = min([y_pred_sort[x] for x in one_predictions])

print(f"CÓMO PREDECIR CON ESTE MODELO:\n\n Paso 1.- Calcularlo con la regresión lineal.\n PASO 2.- Aplicarle sigmoide para acotarlo al intervalo [0,1].\n PASO 3.- Si el resultado es mayor a {umbral} entonces predecir 1.\n          De lo contrario, predecir 0.\n\nAsegúrense de meter esto en score.py")


umb = json.dumps({"umbral": [umbral]})
file = open("umbral.json", "w")
file.write(umb)
file.close()

file = open('model.pkl', 'wb')
pickle.dump(reg, file)
file.close()


CÓMO PREDECIR CON ESTE MODELO:

 Paso 1.- Calcularlo con la regresión lineal.
 PASO 2.- Aplicarle sigmoide para acotarlo al intervalo [0,1].
 PASO 3.- Si el resultado es mayor a 0.5260736508016486 entonces predecir 1.
          De lo contrario, predecir 0.

Asegúrense de meter esto en score.py
